# 1. Importar librerías y configurar entorno
En esta sección se definen las librerías y configuraciones básicas necesarias para el resto del notebook. Se incluyen las importaciones de módulos externos y la recuperación de variables de entorno relevantes.

In [ ]:
import os
import re
import ast
import torch
from tqdm import tqdm
from typing_extensions import TypedDict

from langchain_community.utilities import SQLDatabase
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoTokenizer
)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_toolkits import create_retriever_tool
from langgraph.prebuilt import create_react_agent

# Se asume que la variable de entorno HF_API_KEY ya existe.
api_key = os.environ.get("HF_API_KEY")

# 2. Conexión con la base de datos
Aquí se configura la conexión con la base de datos PostgreSQL mediante `SQLDatabase` de `langchain_community.utilities`. Se utilizan las credenciales para conformar la URI y luego instanciar el objeto `db`.

In [ ]:
usuario = 'postgres'
password = 'place_rag_password'
host = 'localhost'     # o la IP/URL de tu servidor
puerto = '5432'        # puerto por defecto de PostgreSQL
base_datos = 'place_rag_db'

# Crear la URL de conexión
uri = f"postgresql+psycopg2://{usuario}:{password}@{host}:{puerto}/{base_datos}"

db = SQLDatabase.from_uri(uri)

# 3. Definición del Prompt del Sistema
Este prompt sirve para guiar al modelo en la generación de consultas SQL, asegurando que solo utilice los nombres de tablas y columnas existentes en el esquema de la base de datos proporcionada.

In [ ]:
system_prompt = f"""
Dada una pregunta de entrada, crea una consulta de postgresql sintácticamente correcta.
Usa solo los nombres de las columnas que puedes ver en la descripción del esquema.
No consultes columnas que no existen.
Utiliza únicamente las siguientes tablas: 'entidades', 'expedientes', 'paises', 'regiones'
Esquema de la base de datos:
{db.table_info}
"""

# 4. Configurar modelo *Deepseek-coder-1.3b-base*
Se configura el modelo de lenguaje *Deepseek-coder* con carga en 8 bits (quantization) para optimizar memoria. También se define el tokenizer correspondiente.

In [ ]:
model_name = "deepseek-ai/deepseek-coder-1.3b-base"
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

# Deshabilitamos el uso de caché para evitar problemas de compatibilidad
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)

# Ajustes del tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 5. Definir funciones para generación de consulta y respuesta
1. **extraer_query_sql**: Busca dentro del texto la consulta SQL.
2. **generar_consulta_deepseek**: Genera una consulta SQL a partir de la pregunta del usuario utilizando el modelo Deepseek.
3. **generar_respuesta_deepseek**: Ejecuta la consulta generada y devuelve una respuesta en base a los datos obtenidos.

In [ ]:
def extraer_query_sql(texto):
    """
    Busca en el texto una consulta que empiece con SELECT * y termine con ;.
    Devuelve la consulta completa si la encuentra.
    """
    patron = re.compile(
        r"SELECT \*(?:.|\n)*?;"
    )
    consulta = patron.findall(texto)
    return consulta

def generar_consulta_deepseek(consulta_usuario, model, tokenizer):
    """
    Utiliza el prompt del sistema y la pregunta del usuario para generar una consulta SQL.
    Se intenta extraer la consulta final del texto de salida del modelo.
    """
    prompt_text = system_prompt + " Pregunta: " + consulta_usuario + " Comienza la query siempre por SELECT * y termínala siempre por ; Respuesta: SELECT *"
    inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")

    success = False
    outputs = None
    while not success:
        try:
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=256)
            success = True
        except Exception as e:
            print(e)

    ai_msg = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        # Intentamos quedarnos con la segunda aparición si hubiera.
        resultado = extraer_query_sql(ai_msg)[1]
    except:
        # Si no la encontramos, devolvemos directamente todo el mensaje.
        resultado = ai_msg
    return resultado

def generar_respuesta_deepseek(query, model, tokenizer):
    """
    Ejecuta la consulta generada contra la base de datos. Con los resultados, construye un prompt
    para el modelo Deepseek, que finalmente produce la respuesta al usuario.
    """
    resultado_consulta = db.run(query)
    prompt = ("Responde a la siguiente pregunta" + consulta_usuario + 
              "Utilizando estos datos: " + str(resultado_consulta) + 
              "Si no se aportan datos, responde que no hay contratos que se ajusten a la pregunta.")
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    success = False
    outputs = None
    while not success:
        try:
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=256)
            success = True
        except Exception as e:
            print(e)

    ai_msg = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        # Algunos modelos devuelven la respuesta en .content, etc.
        resultado = ai_msg.content
    except:
        resultado = ai_msg if ai_msg else "Error inesperado."
    return resultado

# 6. Probar generación de consulta y respuesta con Deepseek
En esta sección se ejecuta la función de generación de consulta y, con la consulta resultante, se obtiene la respuesta correspondiente del modelo.

In [ ]:
# Generación de consulta
consulta_generada = generar_consulta_deepseek(consulta_usuario, model, tokenizer)
print("Consulta generada:", consulta_generada)

# Generación de respuesta
respuesta = generar_respuesta_deepseek(consulta_generada, model, tokenizer)
print("Respuesta final:", respuesta)

# 7. Configurar modelo OpenAI GPT-4
Se instancia un LLM a través de `ChatOpenAI` apuntando a GPT-4 (simbolizado aquí como `gpt-4o`).

In [ ]:
class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

llm = ChatOpenAI(model="gpt-4o")

# 8. Generar consultas con agente ReAct y GPT-4
Se crea un agente tipo ReAct con las herramientas de la base de datos (`SQLDatabaseToolkit`) y el modelo `llm` (GPT-4). El prompt del sistema se comparte para guiar al agente.

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools

In [ ]:
agent_executor = create_react_agent(llm, tools, prompt=system_prompt)

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": consulta_usuario}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

# 9. Mejorar la precisión de las consultas mediante *embeddings*
Se utiliza la capacidad de embeddings para encontrar variantes de nombres propios y así garantizar consultas más precisas a la base de datos, especialmente al filtrar por localidad, entidad o región.

In [ ]:
def query_as_list(db, query):
    """
    Ejecuta una consulta y la transforma en una lista con valores únicos,
    limpiando valores numéricos y espacios.
    """
    res = db.run(query)
    # Convertimos la respuesta en una lista plana
    res = [el for sub in ast.literal_eval(str(res)) for el in sub if el]
    # Eliminamos posibles números sueltos y limpiamos espacios
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))

# Obtenemos listas de entidades y regiones
entidades = query_as_list(db, "SELECT name FROM entidades")
regiones = query_as_list(db, "SELECT country_subentity_name FROM regiones")

# Creamos un vector store en memoria
vector_store = InMemoryVectorStore(OpenAIEmbeddings())

# Añadimos textos al vector store
vector_store.add_texts(entidades + regiones)

# Creamos el retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Úselo para buscar valores para filtrar. La entrada es una ortografía aproximada "
    "del nombre propio, la salida son nombres propios válidos. Utilice el sustantivo más "
    "similar a la búsqueda."
)

retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

# Sufijo para guiar al sistema en el uso del retriever
suffix = (
    "Si necesita filtrar por un nombre propio como el de una entidad o región, SIEMPRE debes buscar primero "
    "el valor del filtro usando la herramienta 'search_proper_nouns'. No intentes "
    "adivinar el nombre propio; utiliza esta función para encontrar nombres similares"
)

# Unimos prompt del sistema y sufijo
system = f"{system_prompt}\n\n{suffix}"

# Añadimos el retriever_tool a la lista de herramientas
tools.append(retriever_tool)

# 10. Agente ReAct con embeddings
Se crea un nuevo agente ReAct que, además de las herramientas de la base de datos, utiliza la herramienta de recuperación de nombres propios para encontrar la ortografía correcta de entidades o regiones.

In [ ]:
agent = create_react_agent(llm, tools, prompt=system)
consulta_usuario = "¿Tienes información sobre licitaciones publicadas en Vurgos?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": consulta_usuario}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()